# Basic Examples

In [ ]:
import numpy as np

# This file uses multiple Sessions, but only one will be open at a time, unless MULTI_SESSION is set to True
MULTI_SESSION = False
session_storage = dict()

from example_data import *

sr = 48000
annotations = np.genfromtxt(simple_annotations_file, delimiter=',')
audio, _ = lfb.read_audio(simple_audio_file, mono=True, Fs=sr)

In [ ]:
# example 1:
# very simple example plotting function plots
# audio is a numpy array containing audio at the sampling rate sr

from makeplotplayable import Session

# crate a Session from the audio data
session = Session(audio, sr)


#decorate the plot function with the session
# noinspection PyShadowingNames
@session
def plot(audio, sr, name):
    # define a plot function, which does not use any global (not even imports)
    # this is a limitation from multiprocessing
    # everything the function needs should be parsed as an argument/key-word-argument
    # simple datatypes definitely work, we can even pass other functions (stay away from lambdas)
    # if there are issues check the limitations of dill
    import libfmp.b as lfb

    fig, ax, _ = lfb.plot_signal(audio, sr)

    # return the figure, the axis and an optional config dict for the cursor as a tuple
    return fig, ax, {'title': name}


#call the plot function, it will create the plot in another process
plot(audio, sr, "Fig 1")

#we can create multiple plots in on session (all plots in a session will be linked)
plot(audio, sr, "Fig 2")

#start the session to enable audio playback
session.start()

if MULTI_SESSION:
    session_storage["example 1"] = session

In [ ]:
#example 2
# this example shows an alternative way to wrap the plotting function and how the original can be used too.
# And the two alternative return types of the plotting function
# In addition, the way how to stream audio directly from a file is shown

from makeplotplayable import Session


# noinspection PyShadowingNames
def plot(annotations, disable_cursor=False):
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots()

    ax.plot(annotations)

    # shows that an external plot doesn't need to be interactive
    if disable_cursor:
        return

    return fig, ax


# create session from file
session = Session.from_file(simple_audio_file, looping=True)
#place curser at 10s
session.time = 10

#start the session to enable audio playback
session.start()

#call the plot function plot will be show normally
plot(annotations)

# wrap the plot function to create a playable version
plot_wrap = session(plot)
# call that playable version
plot_wrap(annotations)
# call playable version with playback disabled
plot_wrap(annotations, disable_cursor=True)

if MULTI_SESSION:
    session_storage["example 2"] = session

In [ ]:
# It is possible to query the current playback time and state
print(f'Session state: {session.time=} {session.paused=}')

In [ ]:
# example 3
# this example shows that multiple plotting functions are possible.
# How to use extra functions.
# The way the "sparse" mapping works (mapping as a list of time position pairs)
# and additional plot parameters
from makeplotplayable import Session

# create session from file
session = Session(audio, sr)
session.start()


# noinspection PyShadowingNames
@session
def plotA(audio, sr):
    import libfmp.b as lfb

    fig, ax, line = lfb.plot_signal(audio, sr)

    return fig, ax


# noinspection PyShadowingNames
@session
def plotB(audio, sr, mapper):
    import libfmp.b as lfb

    fig, ax, line = lfb.plot_signal(audio, sr)

    mapping = mapper(audio.shape[0] / sr)

    return fig, ax, {"mapping": mapping,
                     "axvline_kwargs": {"alpha": 0.9, "ls": 'dashdot', "color": 'c', "lw": 2, "zorder": 10},
                     "title": "Sparse Mapping Function",
                     "window_pos": (0, 0)}


# additional function has same restrictions/requirements (see imports in function)
# noinspection PyShadowingNames
def mapper(duration):
    import numpy as np
    ## mapping the midpoint offset
    ## after half the audio is played the curser is still at the 10% mark
    mapping = np.array([
        [0, 0],
        [duration / 2, duration / 10],
        [duration, duration]
    ])
    return mapping


# it is possible to use different plotting functions
plotA(audio, sr)
plotB(audio, sr, mapper)

if MULTI_SESSION:
    session_storage["example 3"] = session

In [ ]:
# example 4
# this example shows an alternative mapping methode
# The way the "dense" mapping works (mapping as a list of all times between features)
from makeplotplayable import Session

# create session from file
session = Session(audio, sr)
session.start()


# noinspection PyShadowingNames
@session
def plotA(audio, sr):
    import libfmp.b as lfb

    fig, ax, line = lfb.plot_signal(audio, sr)

    return fig, ax


# noinspection PyShadowingNames
@session
def plotB(annotations):
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots()

    ax.plot(annotations[:, 0])
    print(annotations.shape)
    mapping = annotations[:, 0], 0, annotations.shape[0] - 1



    return fig, ax, {"mapping": mapping,
                     "title": "Dense Mapping"}

plotA(audio, sr)
plotB(annotations)

if MULTI_SESSION:
    session_storage["example 4"] = session

In [ ]:
# example 5:
# this example shows how an extra axis for a cursor could be constructed
# this example shows no new core functionality

session = Session(audio, sr)
session.start()


# noinspection PyShadowingNames
@session
def plot(audio, sr, annotations):
    import matplotlib.pyplot as plt
    from matplotlib.transforms import Bbox
    import libfmp.b as lfb

    fig, axs = plt.subplots(2)

    lfb.plot_signal(audio, sr, ax=axs[0])
    axs[1].plot(annotations)
    axs[1].set_xlim(annotations[0, 0], annotations[-1, 0])

    fig.tight_layout()

    pa = axs[0].get_position().get_points()
    pb = axs[1].get_position().get_points()

    p = Bbox([[pb[0, 0], pb[0, 1]], [pa[1, 0], pa[1, 1]]])
    ax = fig.add_axes(p, xlim=(0, 1))
    ax.update_datalim([(0, 0), (1, 0)])
    ax.axis('off')

    return fig, ax, {"title": "Subplot Cursor"}


plot(audio, sr, annotations)

if MULTI_SESSION:
    session_storage["example 5"] = session

In [ ]:
# example 6:
# this example shows how extra elements can be animated and how to register mpl events

from makeplotplayable import Session

session = Session(audio, sr)
session.start()


# noinspection PyShadowingNames
@session
def plot(x, sr):
    import libfmp.b as lfb

    fig, ax, line = lfb.plot_signal(x, sr)

    text = ax.text(10, 0, '____', style='italic', bbox={
        'facecolor': 'red', 'alpha': 0.5, 'pad': 10})

    string = ""
    force_redraw = False

    # a simple demo of the mpl key_press_event; write into a text box
    def on_key(event):
        # raise Exception("sfasdf")
        # nonlocal allows to override variables from the outer functions scope similar to global
        nonlocal string, force_redraw
        force_redraw = True
        if event.key == "delete":
            string = ""
        elif event.key == "backspace":
            if len(string) > 0:
                string = string[:-1]
        else:
            string += event.key

    fig.canvas.mpl_connect('key_press_event', on_key)

    last_draw_input = None

    def draw_function(time, pos, paused):
        nonlocal last_draw_input, force_redraw
        # only redraw if something animated changed
        if last_draw_input == (time, pos, paused) and not force_redraw:
            return False

        last_draw_input = (time, pos, paused)

        text.set_text(f"{time=:3.2f}, {pos=:3.2f}, {paused=} ♯ \N{Music Flat Sign} {string}")

        force_redraw = False
        return True

    return fig, ax, {"title": "Extra Animations",
                     "draw_function": draw_function,
                     "artists": [text]}


plot(audio, sr)

if MULTI_SESSION:
    session_storage["example 6"] = session

In [ ]:
# example 7:
# very simple example streaming the audio file from a http(s) url

from makeplotplayable import Session

# crate a Session from the audio data from an url
session = Session.from_file(url_audio_example)

# noinspection PyShadowingNames
@session
def plot(annotations, title):
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots()
    ax.plot(annotations)

    return fig, ax, {"title": title}

plot(annotations, "Web example")

session.start()

if MULTI_SESSION:
    session_storage["example 7"] = session